In [53]:
import copy;
import requests;
from bs4 import BeautifulSoup;

def getSoup(inurl):
    r = requests.get(inurl)
    r.encoding = 'utf-8'
    soup=BeautifulSoup(r.text);
    return soup;
#end of getSoup

def gen_page_number_list():
    result=list();
    magical_number=21+1;#this is found by trying 
    for i in range(1,22):
        sup=getSoup("https://movies.yahoo.com.tw/moviegenre_result.html?genre_id="+str(i)+"&page=1");
        page_count=1;
        if sup.find("div",class_="page_numbox") is not None:
            lis=sup.find("div",class_="page_numbox").find_all("li");
            page_count=int(lis[len(lis)-3].text);
        #print({"id":i,"page_count":page_count});
        result.append({"genre_id":i,"page_count":page_count});
    return result;
#end of gen_page_number_list

def extend_page_dataset(indataset,inurl):
    a_data={
        "name":"",
        "link":"",
        "when":"",
        "desc":"",
        "class":[]
    };
    sup=getSoup(inurl);
    info_part=sup.find("div",class_="movie_intro_info_r");
    desc_str=sup.find("span",id="story").text.strip();
    
    when_str="";
    for entry in info_part.find_all("span"):
        if entry.text.startswith("上映日期："):
            when_str=copy.deepcopy(entry.text);
            when_str=when_str[len("上映日期："):];
            break;
    if when_str=="":
        raise Exception("date not found");
        
    class_list=[];
    for entry in info_part.find("div",class_="level_name_box").find_all("a"):
        class_list.append(entry.text.strip());
    
    
    a_data["name"]=info_part.find("h1").text;
    a_data["link"]=inurl;
    a_data["when"]=when_str;
    a_data["desc"]=desc_str;
    a_data["class"]=class_list;
    
    indataset.append(a_data);
    return indataset;
#end of extend_page_dataset


    